In [1]:
from structures import *
from utils import *
import random

In [2]:
class Node:
    def __init__(self):
        self.parent = None
        self.leaf = True
        self.keys = []
        self.pointers = [None]
        self.max_keys = 3
        self.min_leaf_keys = (self.max_keys + 1) // 2
        self.min_non_leaf_keys = self.max_keys // 2

In [3]:
_="""
We will assume, that the key is not present into the list
Start from root node, perform exact match for key as ‘key’ till a leaf node. Let the search path
be x1, x2, … , xh. The x1 is first node so root, then xh is leaf node. Each node xi is parent of xi+1
Insert the new object where key is ‘key’, and value is v into xh.
i := h
while xi overflows, do
   divide xi into two nodes, by moving the larger half of the keys into a new node p.
   if xi is leaf node, link p into the linked list among leaf nodes.
   identify a key k, to be inserted into the parent level along with child pointer pointing p.
   The choice of k depends on the type of the node xi. If xi is leaf node, we will perform
   copy up. So smallest key in p, is copied as k to the parent level. On the other hand, if xi is
   non-leaf node, then we will perform push up. So smallest key in p, will be copied into k,
   in the parent node.
   if i = 0, then
      create a new index node as the new root. In the new root store node with key k,
      and two child xi and p.
      return
   else
      insert a key k and a child pointer pointing to p, into node xi-1.
      i := i – 1
   end if
done
"""

In [4]:
def get_right_sibling(node):
    if node.parent == None:
        return None
    for i in range(len(node.parent.pointers)-1): # skip the last as if node is last pointer then no right sibling
        if node.parent.pointers[i] is node:
            return node.parent.pointers[i+1]
    return None

In [5]:
def get_left_sibling(node):
    if node.parent == None:
        return None
    for i in range(1, len(node.parent.pointers)): # skip the first as if node is first pointer then no left sibling
        if node.parent.pointers[i] is node:
            return node.parent.pointers[i-1]
    return None

In [6]:
def delete(node, key):
    delete_helper(node, key)
    if node.pointers[0] == None:
        print("tree is empty")
        return node
    if len(node.keys) == 0:
        print("root has no keys, updating root!")
        node.pointers[0].parent = None
        return node.pointers[0]
    return node

In [7]:
def delete_helper(node, key):
    if node.leaf:
        for i in range(len(node.keys)):
            if node.keys[i] == key:
                node.keys.pop(i)
                node.pointers.pop(i)
                next_largest = node.keys[i] if i < len(node.keys) else None
                break
        
        if next_largest == None:
            next_largest = node.pointers[-1].keys[0] if node.pointers[-1] else None
        
        if node.parent == None:
            return False, next_largest
        
        if len(node.keys) >= node.min_leaf_keys:
            return False, next_largest
        
        print("Leaf underflow")
        # check if can borrow from left sibling
        left_sibling = get_left_sibling(node)
        if left_sibling and len(left_sibling.keys) > node.min_leaf_keys:
            print("Leaf borrow from left")
            leaf_distribute(left_sibling, node)
            return False, next_largest
        
        # check if can borrow from right sibling
        right_sibling = get_right_sibling(node)
        if right_sibling and len(right_sibling.keys) > node.min_leaf_keys:
            print("Leaf borrow from right")
            leaf_distribute(node, right_sibling)
            return False, next_largest
        
        # check if can merge with left sibling
        if left_sibling:
            print("Leaf merge with left")
            leaf_merge(left_sibling, node)
            return True, next_largest
        
        # check if can merge with right sibling
        if right_sibling:
            print("Leaf merge with right")
            leaf_merge(node, right_sibling)
            return True, next_largest
        
        raise Exception("Leaf deletion underflow could never borrow nor merge")
            
    elif not node.leaf:
        deleted = False
        for i in range(len(node.keys)):
            if key < node.keys[i]:
                pos = i
                res = delete_helper(node.pointers[i], key)
                deleted = True
                break
        
        if not deleted:
            pos = len(node.pointers) - 1
            res = delete_helper(node.pointers[-1], key)
        
        if res[0] == False or len(node.keys) >= node.min_non_leaf_keys or node.parent == None:
            replace_key(node, key, res[1])
            return False, res[1]
            
        print("Non leaf underflow")
        # check if can borrow from left sibling
        left_sibling = get_left_sibling(node)
        if left_sibling and len(left_sibling.keys) > node.min_non_leaf_keys:
            print("Non leaf borrow from left")
            distribute(left_sibling, node)
            replace_key(node, key, res[1])
            return False, res[1]
        
        right_sibling = get_right_sibling(node)
        if right_sibling and len(right_sibling.keys) > node.min_non_leaf_keys:
            print("Non leaf borrow from right")
            distribute(node, right_sibling)
            replace_key(node, key, res[1])
            return False, res[1]
        
        # check if can merge with left sibling
        if left_sibling:
            print("Non leaf merge with left")
            merge_with_left(left_sibling, node)
            replace_key(node, key, res[1])
            return True, res[1]
        
        # check if can merge with right sibling
        if right_sibling:
            print("Non leaf merge with right")
            merge_with_right(node, right_sibling)
            replace_key(node, key, res[1])
            return True, res[1]
        

        raise Exception("Non leaf deletion underflow could never borrow nor merge")
            
        

In [8]:
def replace_key(node, old, new):
    if new == None:
        return
    for i in range(len(node.keys)):
        if node.keys[i] == old:
            node.keys[i] = new

In [9]:
def leaf_distribute(left, right):
#     print("leaf_distribute")
    all_keys = left.keys + right.keys
    all_pointers = left.pointers[:-1] + right.pointers[:-1]
    num_left = (len(all_keys) + 1) // 2

    left.keys = all_keys[:num_left]
    left.pointers = all_pointers[:num_left] + [left.pointers[-1]]

    right.keys = all_keys[num_left:]
    right.pointers = all_pointers[num_left:] + [right.pointers[-1]]
    
    update_parent_lb(left)
    update_parent_lb(right)

In [10]:
def update_parent_lb(node):
    for i in range(1, len(node.parent.pointers)):
        if node.parent.pointers[i] is node:
            node.parent.keys[i-1] = node.keys[0]

In [11]:
def leaf_merge(left, right):
#     print("leaf_merge")
    left.keys.extend(right.keys)
    left.pointers.pop()
    left.pointers.extend(right.pointers)
    remove_from_parent_next_pointer_and_key(left)
    update_parent_lb(left)

In [12]:
def remove_from_parent_next_pointer_and_key(node):
    for i in range(len(node.parent.pointers)-1):
        if node.parent.pointers[i] is node:
            node.parent.pointers.pop(i+1)
            return node.parent.keys.pop(i)
        
def remove_from_parent_prev_pointer_and_key(node):
    for i in range(1, len(node.parent.pointers)):
        if node.parent.pointers[i] is node:
            node.parent.pointers.pop(i-1)
            return node.parent.keys.pop(i-1)

In [13]:
def distribute(left, right):
#     print("distribute")
    parent = left.parent
    pivot_pos = None
    for i in range(len(parent.pointers)):
        if parent.pointers[i] is left:
            pivot_pos = i
    if pivot_pos == None or pivot_pos == len(parent.pointers) - 1:
        print("pivot_pos:", pivot_pos)
        raise Exception("If left has a right sibling, it must have a pivot_pos < len(parent.pointers) - 1")
    assert parent.pointers[pivot_pos+1] is right
    
    num_left = (len(right.keys) + len(left.keys) + 1) // 2
    num_right = (len(right.keys) + len(left.keys)) - num_left
    
    for _ in range(num_left - len(left.keys)):
        left.keys.append(parent.keys[pivot_pos])
        parent.keys[pivot_pos] = right.keys.pop(0)
        left.pointers.append(right.pointers.pop(0))
        left.pointers[-1].parent = left
        left.keys[-1] = left.pointers[-1].keys[0]
    
    for _ in range(num_right - len(right.keys)):
        right.keys.insert(0, parent.keys[pivot_pos])
        parent.keys[pivot_pos] = left.keys.pop()
        right.pointers.insert(0, left.pointers.pop())
        right.pointers[0].parent = right


In [14]:
def merge_with_right(node, right):
    x = remove_from_parent_next_pointer_and_key(node)
    node.keys.append(x)
    for i in range(len(right.pointers)):
        node.pointers.append(right.pointers[i])
        node.pointers[-1].parent = node
        node.keys[-1] = node.pointers[-1].keys[0]
        if i == len(right.keys):
            break
        node.keys.append(right.keys[i])

def merge_with_left(left, node):
    x = remove_from_parent_prev_pointer_and_key(node)
    node.keys.insert(0, x)
    # right.keys[0] = right.pointers[0].keys[0]
    while left.pointers:
        node.pointers.insert(0, left.pointers.pop())
        node.pointers[0].parent = node
        if left.keys:
            node.keys.insert(0, left.keys.pop())

In [15]:
_="""
We will assume, that the key is present into the list
Start from root node, perform exact match for key as ‘key’ till a leaf node. Let the search path
be x1, x2, … , xh. The x1 is first node so root, then xh is leaf node. Each node xi is parent of xi+1
delete the object where key is ‘key’ from xh.
if h = 1, then return, as there is only one node which is root.
i := h
while xi underflows, do
   if immediate sibling node s of xi, has at least m/2 + 1 elements, then
      redistribute entries evenly between s and xi.
      corresponding to redistribution, a key k in the parent node xi-1, will be changed.
      if xi is non-leaf node, then
         k is dragged down to xi. and a key from s is pushed up to fill the place of k
      else
         k is simply replaced by a key in s
      return
   else
      merge xi with the sibling node s. Delete the corresponding child pointer in xi-1.
      if xi is an internal node, then
         drag the key in xi-1. which is previously divides xi and s. into the new node
         xi and s, into the new node xi.
      else
         delete that key in xi-1.
      i := i – 1
   end if
done
"""

In [16]:
def insert_helper(node, key, value):
    if node.leaf:
        inserted = False
        for i in range(len(node.keys)):
            if key < node.keys[i]:
                node.keys.insert(i, key)
                node.pointers.insert(i, value)
                inserted = True
                break
        if not inserted:
            node.pointers.insert(len(node.keys), value)
            node.keys.insert(len(node.keys), key)
        if len(node.keys) > node.max_keys:
            num_left = (len(node.keys) + 1) // 2
            
            right_node = Node()
            right_node.keys = node.keys[num_left:]
            right_node.pointers = node.pointers[num_left:-1] + [None]
            
            node.keys = node.keys[:num_left]
            node.pointers = node.pointers[:num_left] + [right_node]
            
            to_insert = Node()
            to_insert.leaf = False
            to_insert.keys = [right_node.keys[0]]
            to_insert.pointers = [node, right_node]
            
            node.parent = to_insert
            right_node.parent = to_insert
            
            return to_insert
        return None
        
    elif not node.leaf:
        inserted = False
        for i in range(len(node.keys)):
            if key < node.keys[i]:
                pos = i
                res = insert_helper(node.pointers[i], key, value)
                inserted = True
                break
        if not inserted:
            pos = len(node.pointers) - 1
            res = insert_helper(node.pointers[-1], key, value)
        if res == None:
            return None
        node.keys.insert(pos, res.keys[0])
        node.pointers[pos] = res.pointers[0]
        node.pointers.insert(pos+1, res.pointers[1])
        node.pointers[pos].parent = node
        node.pointers[pos+1].parent = node
        
        if len(node.keys) > node.max_keys:
            num_left = len(node.keys) // 2
            
            right_node = Node()
            right_node.leaf = False
            right_node.keys = node.keys[num_left+1:]
            right_node.pointers = node.pointers[num_left+1:]
            for pointer in right_node.pointers:
                pointer.parent = right_node
            
            to_insert = Node()
            to_insert.leaf = False
            to_insert.keys = [node.keys[num_left]]
            to_insert.pointers = [node, right_node]
            
            node.keys = node.keys[:num_left]
            node.pointers = node.pointers[:num_left+1]
            
            node.parent = to_insert
            right_node.parent = to_insert
            
            return to_insert
        return None

In [17]:
def insert(root, key, value):
    res = insert_helper(root, key, value)
    return res if res else root

In [18]:
def show(root):
    cur = [root]
    while cur:
        nxt = []
        to_print = []
        for node in cur:
            if node == None:
                print("Empty tree")
                return
            for key in node.keys:
                to_print.append(key)
            to_print.append("|")
            for pointer in node.pointers:
                if type(pointer) != Node:
                    break
                nxt.append(pointer)
        print(" ".join(str(x) for x in to_print))
        print()
        cur = nxt

In [19]:
def validate_parent(root):
    for p in root.pointers:
        if type(p) == Node:
            assert p.parent is root
            validate_parent(p)
        else:
            return

In [20]:
def validate(root):
    if root.parent != None:
        if root.leaf:
            assert root.min_leaf_keys <= len(root.keys) <= root.max_keys
        else:
            assert root.min_non_leaf_keys <= len(root.keys) <= root.max_keys
    for i in range(len(root.keys)-1):
        assert root.keys[i] < root.keys[i+1]
    if type(root.pointers[0]) != Node:
        return root.pointers[0]
    for i in range(len(root.pointers)-1):
        assert root.pointers[i].keys[0] < root.pointers[i+1].keys[0]
    for i in range(len(root.pointers)):
        if i > 0:
            assert root.keys[i-1] == validate(root.pointers[i])
        else:
            validate(root.pointers[i])
    return validate(root.pointers[0])

In [22]:
while True:
    root = Node()
    arr = list(range(24))
    random.shuffle(arr)
    print("insertion:", arr)
    for i in arr:
        root = insert(root, i, i)
        validate(root)
        validate_parent(root)
    random.shuffle(arr)
    print("deletion:", arr)
    for i in arr:
        print("Delete:", i)
        root = delete(root, i)
        show(root)
        validate(root)
        validate_parent(root)

insertion: [15, 22, 6, 11, 2, 14, 10, 18, 4, 5, 21, 9, 13, 7, 16, 8, 17, 20, 3, 12, 23, 0, 1, 19]
deletion: [22, 7, 16, 1, 0, 4, 13, 10, 17, 2, 11, 15, 18, 3, 9, 21, 6, 20, 5, 8, 23, 19, 14, 12]
Delete: 22
9 15 |

2 4 6 | 11 13 | 17 19 21 |

0 1 | 2 3 | 4 5 | 6 7 8 | 9 10 | 11 12 | 13 14 | 15 16 | 17 18 | 19 20 | 21 23 |

Delete: 7
9 15 |

2 4 6 | 11 13 | 17 19 21 |

0 1 | 2 3 | 4 5 | 6 8 | 9 10 | 11 12 | 13 14 | 15 16 | 17 18 | 19 20 | 21 23 |

Delete: 16
Leaf underflow
Leaf merge with right
9 15 |

2 4 6 | 11 13 | 19 21 |

0 1 | 2 3 | 4 5 | 6 8 | 9 10 | 11 12 | 13 14 | 15 17 18 | 19 20 | 21 23 |

Delete: 1
Leaf underflow
Leaf merge with right
9 15 |

4 6 | 11 13 | 19 21 |

0 2 3 | 4 5 | 6 8 | 9 10 | 11 12 | 13 14 | 15 17 18 | 19 20 | 21 23 |

Delete: 0
9 15 |

4 6 | 11 13 | 19 21 |

2 3 | 4 5 | 6 8 | 9 10 | 11 12 | 13 14 | 15 17 18 | 19 20 | 21 23 |

Delete: 4
Leaf underflow
Leaf merge with left
9 15 |

6 | 11 13 | 19 21 |

2 3 5 | 6 8 | 9 10 | 11 12 | 13 14 | 15 17 18 | 19 20 | 21 2


Delete: 11
Leaf underflow
Leaf merge with left
16 |

3 | 19 |

0 2 | 3 6 12 | 16 17 | 19 21 |

Delete: 6
16 |

3 | 19 |

0 2 | 3 12 | 16 17 | 19 21 |

Delete: 19
Leaf underflow
Leaf merge with left
Non leaf underflow
Non leaf merge with left
root has no keys, updating root!
3 16 |

0 2 | 3 12 | 16 17 21 |

Delete: 2
Leaf underflow
Leaf merge with right
16 |

0 3 12 | 16 17 21 |

Delete: 16
17 |

0 3 12 | 17 21 |

Delete: 21
Leaf underflow
Leaf borrow from left
12 |

0 3 | 12 17 |

Delete: 17
Leaf underflow
Leaf merge with left
root has no keys, updating root!
0 3 12 |

Delete: 3
0 12 |

Delete: 12
0 |

Delete: 0
tree is empty
|

insertion: [21, 11, 22, 14, 1, 18, 17, 9, 15, 7, 6, 20, 12, 8, 4, 16, 3, 0, 5, 19, 13, 10, 2, 23]
deletion: [18, 2, 0, 13, 16, 3, 15, 6, 11, 14, 17, 21, 10, 20, 8, 22, 7, 5, 4, 9, 1, 12, 23, 19]
Delete: 18
Leaf underflow
Leaf merge with right
9 17 |

2 4 7 | 12 14 | 21 |

0 1 | 2 3 | 4 5 6 | 7 8 | 9 10 11 | 12 13 | 14 15 16 | 17 19 20 | 21 22 23 |

Delete: 2
L

Leaf merge with right
Non leaf underflow
Non leaf merge with left
6 13 |

2 4 | 8 | 15 18 |

0 1 | 2 3 | 4 5 | 6 7 | 8 10 12 | 13 14 | 15 16 17 | 18 22 23 |

Delete: 22
6 13 |

2 4 | 8 | 15 18 |

0 1 | 2 3 | 4 5 | 6 7 | 8 10 12 | 13 14 | 15 16 17 | 18 23 |

Delete: 8
6 13 |

2 4 | 10 | 15 18 |

0 1 | 2 3 | 4 5 | 6 7 | 10 12 | 13 14 | 15 16 17 | 18 23 |

Delete: 7
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf borrow from left
4 13 |

2 | 6 | 15 18 |

0 1 | 2 3 | 4 5 | 6 10 12 | 13 14 | 15 16 17 | 18 23 |

Delete: 23
Leaf underflow
Leaf borrow from left
4 13 |

2 | 6 | 15 17 |

0 1 | 2 3 | 4 5 | 6 10 12 | 13 14 | 15 16 | 17 18 |

Delete: 6
4 13 |

2 | 10 | 15 17 |

0 1 | 2 3 | 4 5 | 10 12 | 13 14 | 15 16 | 17 18 |

Delete: 5
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf borrow from right
4 15 |

2 | 13 | 17 |

0 1 | 2 3 | 4 10 12 | 13 14 | 15 16 | 17 18 |

Delete: 18
Leaf underflow
Leaf merge with left
Non leaf underflow
Non leaf merge with left
4 |


Delete: 22
7 14 |

5 | 11 | 19 |

1 2 3 | 5 6 | 7 10 | 11 12 | 14 16 17 | 19 20 |

Delete: 20
Leaf underflow
Leaf borrow from left
7 14 |

5 | 11 | 17 |

1 2 3 | 5 6 | 7 10 | 11 12 | 14 16 | 17 19 |

Delete: 3
7 14 |

5 | 11 | 17 |

1 2 | 5 6 | 7 10 | 11 12 | 14 16 | 17 19 |

Delete: 7
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf merge with left
14 |

5 10 | 17 |

1 2 | 5 6 | 10 11 12 | 14 16 | 17 19 |

Delete: 1
Leaf underflow
Leaf merge with right
14 |

10 | 17 |

2 5 6 | 10 11 12 | 14 16 | 17 19 |

Delete: 17
Leaf underflow
Leaf merge with left
Non leaf underflow
Non leaf merge with left
root has no keys, updating root!
10 14 |

2 5 6 | 10 11 12 | 14 16 19 |

Delete: 19
10 14 |

2 5 6 | 10 11 12 | 14 16 |

Delete: 12
10 14 |

2 5 6 | 10 11 | 14 16 |

Delete: 14
Leaf underflow
Leaf merge with left
10 |

2 5 6 | 10 11 16 |

Delete: 11
10 |

2 5 6 | 10 16 |

Delete: 10
Leaf underflow
Leaf borrow from left
6 |

2 5 | 6 16 |

Delete: 2
Leaf underflow
Leaf merge with r

Delete: 3
Leaf underflow
Leaf merge with right
17 |

14 | 20 |

2 4 7 | 14 15 | 17 19 | 20 21 |

Delete: 19
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf merge with left
root has no keys, updating root!
14 17 |

2 4 7 | 14 15 | 17 20 21 |

Delete: 20
14 17 |

2 4 7 | 14 15 | 17 21 |

Delete: 2
14 17 |

4 7 | 14 15 | 17 21 |

Delete: 4
Leaf underflow
Leaf merge with right
17 |

7 14 15 | 17 21 |

Delete: 7
17 |

14 15 | 17 21 |

Delete: 21
Leaf underflow
Leaf merge with left
root has no keys, updating root!
14 15 17 |

Delete: 17
14 15 |

Delete: 15
14 |

Delete: 14
tree is empty
|

insertion: [8, 9, 11, 17, 12, 23, 22, 3, 19, 4, 0, 1, 18, 16, 21, 6, 14, 5, 10, 2, 7, 15, 20, 13]
deletion: [17, 13, 14, 5, 3, 16, 12, 6, 11, 2, 7, 23, 1, 18, 0, 20, 8, 22, 10, 21, 19, 15, 9, 4]
Delete: 17
Leaf underflow
Leaf borrow from right
11 18 |

3 5 8 | 14 | 20 22 |

0 1 2 | 3 4 | 5 6 7 | 8 9 10 | 11 12 13 | 14 15 16 | 18 19 | 20 21 | 22 23 |

Delete: 13
11 18 |

3 5 8 | 14 | 20 22 

...

Delete: 7
tree is empty
|

insertion: [19, 20, 4, 16, 12, 10, 18, 8, 5, 11, 15, 9, 17, 13, 14, 2, 23, 3, 7, 0, 1, 6, 21, 22]
deletion: [1, 22, 18, 0, 19, 20, 2, 8, 4, 6, 17, 12, 10, 14, 7, 3, 11, 5, 15, 9, 13, 21, 23, 16]
Delete: 1
Leaf underflow
Leaf merge with right
8 12 16 |

4 6 | 10 | 14 | 19 21 |

0 2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 18 | 19 20 | 21 22 23 |

Delete: 22
8 12 16 |

4 6 | 10 | 14 | 19 21 |

0 2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 18 | 19 20 | 21 23 |

Delete: 18
8 12 16 |

4 6 | 10 | 14 | 19 21 |

0 2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 | 19 20 | 21 23 |

Delete: 0
8 12 16 |

4 6 | 10 | 14 | 19 21 |

2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 | 19 20 | 21 23 |

Delete: 19
Leaf underflow
Leaf merge with left
8 12 16 |

4 6 | 10 | 14 | 21 |

2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 20 | 21 23 |

Delete: 20
8 12 16 |

4 6 | 10 | 14 | 21 |

2 3 | 4 5 | 6 7 | 8 9 | 10 11 | 12 13 | 14 15 | 16 17 | 

0 1 2 | 3 4 | 6 7 8 | 9 10 | 11 12 | 13 14 15 | 17 18 | 19 20 23 |

Delete: 20
9 13 |

3 6 | 11 | 17 19 |

0 1 2 | 3 4 | 6 7 8 | 9 10 | 11 12 | 13 14 15 | 17 18 | 19 23 |

Delete: 13
9 14 |

3 6 | 11 | 17 19 |

0 1 2 | 3 4 | 6 7 8 | 9 10 | 11 12 | 14 15 | 17 18 | 19 23 |

Delete: 3
Leaf underflow
Leaf borrow from left
9 14 |

2 6 | 11 | 17 19 |

0 1 | 2 4 | 6 7 8 | 9 10 | 11 12 | 14 15 | 17 18 | 19 23 |

Delete: 23
Leaf underflow
Leaf merge with left
9 14 |

2 6 | 11 | 17 |

0 1 | 2 4 | 6 7 8 | 9 10 | 11 12 | 14 15 | 17 18 19 |

Delete: 8
9 14 |

2 6 | 11 | 17 |

0 1 | 2 4 | 6 7 | 9 10 | 11 12 | 14 15 | 17 18 19 |

Delete: 2
Leaf underflow
Leaf merge with left
9 14 |

6 | 11 | 17 |

0 1 4 | 6 7 | 9 10 | 11 12 | 14 15 | 17 18 19 |

Delete: 7
Leaf underflow
Leaf borrow from left
9 14 |

4 | 11 | 17 |

0 1 | 4 6 | 9 10 | 11 12 | 14 15 | 17 18 19 |

Delete: 1
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf merge with right
14 |

9 11 | 17 |

0 4 6 | 9 10 | 11 12 | 14 15 | 


4 9 | 15 | 20 |

1 3 | 4 6 | 9 11 12 | 13 14 | 15 16 | 18 19 | 20 21 23 |

Delete: 14
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf borrow from left
9 18 |

4 | 13 | 20 |

1 3 | 4 6 | 9 11 12 | 13 15 16 | 18 19 | 20 21 23 |

Delete: 3
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf merge with right
18 |

9 13 | 20 |

1 4 6 | 9 11 12 | 13 15 16 | 18 19 | 20 21 23 |

Delete: 15
18 |

9 13 | 20 |

1 4 6 | 9 11 12 | 13 16 | 18 19 | 20 21 23 |

Delete: 18
Leaf underflow
Leaf borrow from right
19 |

9 13 | 21 |

1 4 6 | 9 11 12 | 13 16 | 19 20 | 21 23 |

Delete: 23
Leaf underflow
Leaf merge with left
Non leaf underflow
Non leaf borrow from left
13 |

9 | 19 |

1 4 6 | 9 11 12 | 13 16 | 19 20 21 |

Delete: 12
13 |

9 | 19 |

1 4 6 | 9 11 | 13 16 | 19 20 21 |

Delete: 9
Leaf underflow
Leaf borrow from left
13 |

6 | 19 |

1 4 | 6 11 | 13 16 | 19 20 21 |

Delete: 20
13 |

6 | 19 |

1 4 | 6 11 | 13 16 | 19 21 |

Delete: 11
Leaf underflow
Leaf merge with left


5 7 | 10 11 | 14 15 17 | 21 22 |

Delete: 21
Leaf underflow
Leaf borrow from left
14 |

10 | 17 |

5 7 | 10 11 | 14 15 | 17 22 |

Delete: 15
Leaf underflow
Leaf merge with right
Non leaf underflow
Non leaf merge with left
root has no keys, updating root!
10 14 |

5 7 | 10 11 | 14 17 22 |

Delete: 22
10 14 |

5 7 | 10 11 | 14 17 |

Delete: 7
Leaf underflow
Leaf merge with right
14 |

5 10 11 | 14 17 |

Delete: 10
14 |

5 11 | 14 17 |

Delete: 11
Leaf underflow
Leaf merge with right
root has no keys, updating root!
5 14 17 |

Delete: 17
5 14 |

Delete: 14
5 |

Delete: 5
tree is empty
|

insertion: [0, 20, 7, 6, 4, 9, 22, 5, 17, 15, 12, 2, 8, 10, 23, 3, 19, 13, 16, 1, 14, 18, 11, 21]
deletion: [17, 8, 1, 3, 23, 0, 6, 11, 16, 9, 14, 10, 20, 19, 4, 13, 12, 7, 18, 5, 15, 21, 22, 2]
Delete: 17
7 15 |

3 5 | 9 12 | 18 20 22 |

0 1 2 | 3 4 | 5 6 | 7 8 | 9 10 11 | 12 13 14 | 15 16 | 18 19 | 20 21 | 22 23 |

Delete: 8
Leaf underflow
Leaf borrow from right
7 15 |

3 5 | 10 12 | 18 20 22 |

0 1 2 |

AssertionError: 